# Only run this first cell once

In [ ]:
!pip install py_midicsv
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import math
import random

# Choose your mid generation model

In [ ]:
#@title Model Name
model_name = "hidude562/Openmusenet-1.5" #@param [] {allow-input: true}

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Now generating the midi from the nn output

#  Generate from scratch (for music prompt go to midi to format single.ipynb for now)

In [ ]:
#@title Default title text
device = torch.device("cuda")
model.cuda()

#generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
#generated = generated.to(device)

def generateMusic(prompt):
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  sample_outputs = model.generate(
                                  generated,
                                  #bos_token_id=random.randint(1,30000),
                                  do_sample=True,
                                  top_k=50,
                                  max_length = 1000,
                                  top_p=1.0,
                                  num_return_sequences=1
                                  )

  #return tokenizer.decode(sample_outputs, skip_special_tokens=True)
  for i, sample_output in enumerate(sample_outputs):
    return tokenizer.decode(sample_output, skip_special_tokens=True)
    print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
  return ""


musicPrompt = "" #@param {type:"string"}
builtInputs = "<|startoftext|>" + musicPrompt
allText = ""
songLen = 1 #@param {type:"integer"}

for i in range(songLen):
  gen = generateMusic(builtInputs[-500:])[len(builtInputs[-500:]):]
  builtInputs += gen
  allText += gen

  finalOutput = builtInputs
  finalOutput = finalOutput[:finalOutput.rfind('|') - 1]
  finalOutput = finalOutput[finalOutput.find('>') + 1:]

print("Output song: ")
print(finalOutput)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output song: 
0$960&60|0$960&64|960$960&67|0$960&66|0$960&62|960$960&57|0$960&67|0$960&64|960$960&60|0$960&69|0$960&66|0&71|0$960&67|960$960&62|0$960&72|0$960&64|960$960&67|0$960&74|0$960&66|960$960&69|0$960&72|0$960&63|960$960&66|0$360&71|0$1140&64|0$1140&62|0$1140&55|360$1140*43|360#360&72|360#360&74|360#360&76|360#360&78|360#360&79|360#360&81|360#360&83|360#360&84|360$360&85|0$360&86|0$480&56|0$480&59|360$480&65|360#360&87|0$360&88|0$480&59|0$480&62|360$480&65|360#360&89|0$360&90|0$480&61|0$480&58|360$480&64|120#360&89|0$360&92|0$480&59|0$480&62|360$480&65|120#360&95|0$360&96|0$480&58|0$480&60|360$480&64|120#360&97|0$360&98|0$360&59|0$360&62|360$480&65|120$360&97|0$360&98|0$480&57|0$480&60|360$480&64|120$360&96|0$360&97|0$480&57|0$480&61|360$480&64|120$360&96|0$360&98|0$480&57|0$480&62|360$480&65|120$360&96|0$360&95|0$480&59|0$480&62|360$480&65|120$360&95|0$360&96|0$480&57|0$480&60|360$480&64|120$360&96|0$360&97|0$480&57|0$480&60|360$480&64|120$360&96|0$360&98|0$360&57|0$480&62|360$

In [ ]:
f = open("test.txt", "w")
f.write(finalOutput + "")
f.close()

# Convert the midi data into a midi

### Midi header things

In [ ]:
midi_tracks = []

def reset_midi():
  global midi_tracks
  midi_tracks = [
  '''0, 0, Header, 1, 7, 480
1, 0, Start_track
1, 0, Title_t, ""
1, 0, Time_signature, 4, 2, 24, 8
1, 0, Tempo, 500000
1, 0, End_track
2, 0, Start_track
2, 0, Control_c, 0, 121, 0
2, 0, Title_t, ""
2, 0, Control_c, 0, 10, 64
2, 0, Control_c, 0, 7, 100
2, 0, Control_c, 0, 11, 127
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 2
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 1
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 38, 0
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 0
2, 0, Control_c, 0, 6, 12
2, 0, Pitch_bend_c, 0, 8192
2, 0, Control_c, 0, 1, 0
2, 0, Program_c, 0, 0'''
  ,
  '''3, 0, Start_track
3, 0, Control_c, 1, 121, 0
3, 0, Title_t, ""
3, 0, Control_c, 1, 10, 64
3, 0, Control_c, 1, 7, 100
3, 0, Control_c, 1, 11, 127
3, 0, Control_c, 1, 101, 0
3, 0, Control_c, 1, 100, 2
3, 0, Control_c, 1, 6, 64
3, 0, Control_c, 1, 101, 0
3, 0, Control_c, 1, 100, 1
3, 0, Control_c, 1, 6, 64
3, 0, Control_c, 1, 38, 0
3, 0, Control_c, 1, 101, 0
3, 0, Control_c, 1, 100, 0
3, 0, Control_c, 1, 6, 12
3, 0, Pitch_bend_c, 1, 8192
3, 0, Control_c, 1, 1, 0
3, 0, Program_c, 1, 40''',

  '''5, 0, Start_track
5, 0, Control_c, 3, 121, 0
5, 0, Title_t, ""
5, 0, Control_c, 3, 10, 64
5, 0, Control_c, 3, 7, 100
5, 0, Control_c, 3, 11, 127
5, 0, Control_c, 3, 101, 0
5, 0, Control_c, 3, 100, 2
5, 0, Control_c, 3, 6, 64
5, 0, Control_c, 3, 101, 0
5, 0, Control_c, 3, 100, 1
5, 0, Control_c, 3, 6, 64
5, 0, Control_c, 3, 38, 0
5, 0, Control_c, 3, 101, 0
5, 0, Control_c, 3, 100, 0
5, 0, Control_c, 3, 6, 12
5, 0, Pitch_bend_c, 3, 8192
5, 0, Control_c, 3, 1, 0
5, 0, Program_c, 3, 1''',

  '''4, 0, Start_track
4, 0, Control_c, 2, 121, 0
4, 0, Title_t, ""
4, 0, Control_c, 2, 10, 64
4, 0, Control_c, 2, 7, 100
4, 0, Control_c, 2, 11, 127
4, 0, Control_c, 2, 101, 0
4, 0, Control_c, 2, 100, 2
4, 0, Control_c, 2, 6, 64
4, 0, Control_c, 2, 101, 0
4, 0, Control_c, 2, 100, 1
4, 0, Control_c, 2, 6, 64
4, 0, Control_c, 2, 38, 0
4, 0, Control_c, 2, 101, 0
4, 0, Control_c, 2, 100, 0
4, 0, Control_c, 2, 6, 12
4, 0, Pitch_bend_c, 2, 8192
4, 0, Control_c, 2, 1, 0
4, 0, Program_c, 2, 60''',

  '''5, 0, Start_track
5, 0, Control_c, 3, 121, 0
5, 0, Title_t, ""
5, 0, Control_c, 3, 10, 64
5, 0, Control_c, 3, 7, 100
5, 0, Control_c, 3, 11, 127
5, 0, Control_c, 3, 101, 0
5, 0, Control_c, 3, 100, 2
5, 0, Control_c, 3, 6, 64
5, 0, Control_c, 3, 101, 0
5, 0, Control_c, 3, 100, 1
5, 0, Control_c, 3, 6, 64
5, 0, Control_c, 3, 38, 0
5, 0, Control_c, 3, 101, 0
5, 0, Control_c, 3, 100, 0
5, 0, Control_c, 3, 6, 12
5, 0, Pitch_bend_c, 3, 8192
5, 0, Control_c, 3, 1, 0
5, 0, Program_c, 3, 1''',
  '''6, 0, Start_track
6, 0, Control_c, 4, 121, 0
6, 0, Title_t, ""
6, 0, Control_c, 4, 10, 64
6, 0, Control_c, 4, 7, 100
6, 0, Control_c, 4, 11, 127
6, 0, Control_c, 4, 101, 0
6, 0, Control_c, 4, 100, 2
6, 0, Control_c, 4, 6, 64
6, 0, Control_c, 4, 101, 0
6, 0, Control_c, 4, 100, 1
6, 0, Control_c, 4, 6, 64
6, 0, Control_c, 4, 38, 0
6, 0, Control_c, 4, 101, 0
6, 0, Control_c, 4, 100, 0
6, 0, Control_c, 4, 6, 12
6, 0, Pitch_bend_c, 4, 8192
6, 0, Control_c, 4, 1, 0
6, 0, Program_c, 4, 66''',
  '''7, 0, Start_track
7, 0, Control_c, 5, 121, 0
7, 0, Title_t, ""
7, 0, Control_c, 5, 10, 64
7, 0, Control_c, 5, 7, 100
7, 0, Control_c, 5, 11, 127
7, 0, Control_c, 5, 101, 0
7, 0, Control_c, 5, 100, 2
7, 0, Control_c, 5, 6, 64
7, 0, Control_c, 5, 101, 0
7, 0, Control_c, 5, 100, 1
7, 0, Control_c, 5, 6, 64
7, 0, Control_c, 5, 38, 0
7, 0, Control_c, 5, 101, 0
7, 0, Control_c, 5, 100, 0
7, 0, Control_c, 5, 6, 12
7, 0, Pitch_bend_c, 5, 8192
7, 0, Control_c, 5, 1, 0
7, 0, Program_c, 5, 3'''
  ]

def add_midi_ends():
  for i in range(len(midi_tracks)):
    midi_tracks[i] += "\n" + str(i + 2) + ", 0, End_track"
def midi_to_string():
  mid_str = ""
  for i in midi_tracks:
    mid_str += i + "\n"
  mid_str += "0, 0, End_of_file"
  return mid_str
reset_midi()

###Run the thing

In [ ]:
reset_midi()

def generate_on_off_events(noteData):
  track_num = 2
  match noteData[3]:
    case "%":
      track_num = 2
    case "^":
      track_num = 3
    case "%":
      track_num = 4
    case "^":
      track_num = 5
    case "&":
      track_num = 6
    case "*":
      track_num = 7
    case "(":
      track_num = 7
    case ")":
      track_num = 7
  #TODO:

  dynamic = 127
  match noteData[3]:
    case "$":
      dynamic = 127
    case "#":
      dynamic = 80
    case "@":
      dynamic = 55
    case "!":
      dynamic = 32

  note_on = str(track_num) + ", " + str(noteData[0]) + ", Note_on_c, 0, " + str(noteData[4]) + ", " + str(dynamic)
  note_off = str(track_num) + ", " + str(noteData[2] + noteData[0]) + ", Note_off_c, 0, " + str(noteData[4]) + ", " + "0"
  return [[noteData[0], note_on, track_num], [noteData[0] + noteData[2], note_off, track_num]]


def test_if_num(num):
  if(num == "-"):
    return True
  try:
    tmp = int(num)
  except:
    return False

  return True

file = open("test.txt", "r").read()
fileTokens = file.split("|")

if(len(fileTokens[-1]) < 5):
  fileTokens.pop()
if(len(fileTokens[0]) < 5):
  fileTokens.pop(0)

allParsedVals = []

timeMarker = 0
for i in range(len(fileTokens)):
  valuesArr = []
  valueBuild = ""
  for j in range(len(fileTokens[i])):
    if(test_if_num(fileTokens[i][j])):
      valueBuild += fileTokens[i][j]
    else:
      if(valueBuild != ""):
        valuesArr.append(int(valueBuild))
      valuesArr.append(fileTokens[i][j])
      valueBuild = ""
  if(valueBuild != ""):
    valuesArr.append(int(valueBuild))
    valueBuild = ""
  try:
    oldTime = valuesArr[0]
  except:
    break
  valuesArr[0] += timeMarker
  allParsedVals.append(valuesArr)
  timeMarker += oldTime

noteEventsArray = []
for val in allParsedVals:
  try:
    events = generate_on_off_events(val)

    noteEventsArray.append(events[0])
    noteEventsArray.append(events[1])
  except Exception as e:
    print("Ai messed up syntax 💀 (still continueing)", e)
    pass
noteEventsArray = sorted(noteEventsArray, key=lambda x: x[0])
#print(noteEventsArray)

outString = ""
for val in noteEventsArray:
  midi_tracks[val[2] - 2] += "\n" + val[1]
  #midi_tracks[val[2][2] - 2] += "\n" +  val[1][1]
add_midi_ends()
outString = midi_to_string()

print(outString)

with open('mididata.csv', 'w') as file:
    file.write(outString)

import py_midicsv as pm

csv_string = "/content/mididata.csv"

# Parse the CSV output of the previous command back into a MIDI file
midi_object = pm.csv_to_midi(csv_string)

# Save the parsed MIDI file to disk
with open("aiOut.mid", "wb") as output_file:
    print("Saved mid successfully")
    midi_writer = pm.FileWriter(output_file)
    midi_writer.write(midi_object)

0, 0, Header, 1, 7, 480
1, 0, Start_track
1, 0, Title_t, ""
1, 0, Time_signature, 4, 2, 24, 8
1, 0, Tempo, 500000
1, 0, End_track
2, 0, Start_track
2, 0, Control_c, 0, 121, 0
2, 0, Title_t, ""
2, 0, Control_c, 0, 10, 64
2, 0, Control_c, 0, 7, 100
2, 0, Control_c, 0, 11, 127
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 2
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 1
2, 0, Control_c, 0, 6, 64
2, 0, Control_c, 0, 38, 0
2, 0, Control_c, 0, 101, 0
2, 0, Control_c, 0, 100, 0
2, 0, Control_c, 0, 6, 12
2, 0, Pitch_bend_c, 0, 8192
2, 0, Control_c, 0, 1, 0
2, 0, Program_c, 0, 0
2, 124, Note_on_c, 0, 42, 127
2, 144, Note_off_c, 0, 42, 0
2, 188, Note_on_c, 0, 68, 127
2, 188, Note_on_c, 0, 73, 127
2, 188, Note_on_c, 0, 77, 127
2, 190, Note_on_c, 0, 46, 127
2, 224, Note_off_c, 0, 68, 0
2, 228, Note_off_c, 0, 73, 0
2, 236, Note_off_c, 0, 77, 0
2, 236, Note_off_c, 0, 46, 0
2, 380, Note_on_c, 0, 42, 127
2, 380, Note_on_c, 0, 38, 127
2, 412, Note_off_c, 0, 42, 0
